In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm
from pathlib import Path

## Load Data

In [2]:
DATA_PATH = Path("data_original")

In [3]:
%%time
users_df = pd.read_csv(DATA_PATH / 'users.csv')
items_df = pd.read_csv(DATA_PATH / 'items.csv')
interactions_df = pd.read_csv(DATA_PATH / 'interactions.csv')

CPU times: total: 1.05 s
Wall time: 2.23 s


In [4]:
items_df = items_df.rename(columns = {'id' : 'item_id'})

## Features

Посмотрим, какие фичи в датасете фильмов являются категориальными и закодируем их с помощью one-hot encoding.

In [5]:
user_cat_feats = ["age", "income", "sex", "kids_flg"]
# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
    # получаем датафрейм с one-hot encoding для каждой категориальной фичи
    ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
    # конкатенируем ohe-hot датафрейм с датафреймом, 
    # который мы получили на предыдущем шаге
    users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,sex_Ж,sex_М,kids_flg_0,kids_flg_1
0,973171,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True
1,962099,True,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False
2,1047345,False,False,False,True,False,False,False,False,False,True,False,False,True,False,True,False
3,721985,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False
4,704055,False,False,True,False,False,False,False,False,False,False,True,False,True,False,True,False


In [6]:
item_cat_feats = ['content_type', 'release_year',
                  'for_kids', 'age_rating', 
                  'studios', 'countries', 'directors']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
    ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
    items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,content_type_film,content_type_series,release_year_1897.0,release_year_1916.0,release_year_1917.0,release_year_1918.0,release_year_1920.0,release_year_1921.0,release_year_1922.0,...,directors_Яннике Систад Якобсен,directors_Янус Мец,directors_Ярив Хоровиц,directors_Ярон Зильберман,directors_Ярополк Лапшин,directors_Ярослав Лупий,"directors_Ярроу Чейни, Скотт Моужер",directors_Ясина Сезар,directors_Ясуоми Умэцу,directors_сения Завьялова
0,10711,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2508,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,10716,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,7868,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,16268,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Текстовые признаки
Добавить текстовые признаки в модель. (3 балла)
TfidfVectorizer

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
for column in ['description', 'keywords']:
    features_clean = items_df[column][items_df[column].notnull()]
    vectorizer = TfidfVectorizer(max_features=500)
    tfidf_m = vectorizer.fit_transform(features_clean.to_list())
    df_vectirized = pd.DataFrame.sparse.from_spmatrix(tfidf_m)
    df_vectirized.columns = [column + '_' + str(x) for x in df_vectirized.columns]
    items_ohe_df = pd.concat([items_ohe_df, df_vectirized], axis = 1)
    

In [9]:
items_ohe_df = items_ohe_df.fillna(0)
items_ohe_df

,item_id,content_type_film,content_type_series,release_year_1897.0,release_year_1916.0,release_year_1917.0,release_year_1918.0,release_year_1920.0,release_year_1921.0,release_year_1922.0,...,keywords_490,keywords_491,keywords_492,keywords_493,keywords_494,keywords_495,keywords_496,keywords_497,keywords_498,keywords_499
0,10711,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2508,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10716,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7868,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16268,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15959,2367,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15960,10632,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15961,4538,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Сделаем матрицу взаимодействий

In [10]:
interactions_df.item_id.value_counts()

item_id
10440    202457
15297    193123
9728     132865
13865    122119
4151      91167
          ...  
8076          1
8954          1
15664         1
818           1
10542         1
Name: count, Length: 15706, dtype: int64

В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил.

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

Можете не фильтровать такие события, тогда у вас будет больше негативных примеров.

In [11]:
interactions_df.user_id.value_counts()

user_id
416206     1341
1010539     764
555233      685
11526       676
409259      625
           ... 
45493         1
615194        1
96848         1
425823        1
697262        1
Name: count, Length: 962179, dtype: int64

In [20]:
print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 10 процентов
interactions_df = interactions_df[interactions_df.watched_pct > 10]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 65974
N items before: 6897

N users after: 65552
N items after: 5303


После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.

In [21]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

65552
5303


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [22]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,watched_pct,uid,iid
0,176549,9506,2021-05-11,4250,72.0,10561,3040
1,699317,1659,2021-05-29,8317,100.0,41858,503
6,1016458,354,2021-08-14,1672,25.0,60634,100
14,5324,8437,2021-04-18,6598,92.0,309,2685
18,927973,9617,2021-06-19,8422,100.0,55430,3065


Отнормируем матрицу взаимодействий и **добавим процент просмотренного в качестве взаимодействия юзеров с айтемами для более репрезентативного сэмплирования. (3 балла)**


In [23]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), 
                             interactions_df.iid.nunique())) 

for i, (user_id, item_id, watched_pct) in enumerate(zip(interactions_df.uid, interactions_df.iid, interactions_df.watched_pct)):
    #то что было до этого
    #interactions_vec[user_id, item_id] += 1
    interactions_vec[user_id, item_id] += watched_pct/100
    

res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]

In [24]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

print(set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique()))

5303
5303
65552
65552
set()


Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [25]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

In [26]:
items_ohe_df

,item_id,content_type_film,content_type_series,release_year_1897.0,release_year_1916.0,release_year_1917.0,release_year_1918.0,release_year_1920.0,release_year_1921.0,release_year_1922.0,...,keywords_490,keywords_491,keywords_492,keywords_493,keywords_494,keywords_495,keywords_496,keywords_497,keywords_498,keywords_499
8,9853,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15,15076,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
16,2904,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
38,1622,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.056351,0.0,0.0,0.0,0.0
40,6677,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15956,1325,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15957,15610,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15958,6443,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
15959,2367,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [27]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid[x])
users_ohe_df = users_ohe_df.set_index("uid")

(тут иногда выскакивает ошибка, ничего менять не нужно просто прогнать селы с 12 еще раз)

In [28]:
users_ohe_df

,user_id,age_age_18_24,age_age_25_34,age_age_35_44,age_age_45_54,age_age_55_64,age_age_65_inf,income_income_0_20,income_income_150_inf,income_income_20_40,income_income_40_60,income_income_60_90,income_income_90_150,sex_Ж,sex_М,kids_flg_0,kids_flg_1
uid,,,,,,,,,,,,,,,,,
43174,721985,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False
16047,269408,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True
22925,384532,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,True
43217,723106,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False
12912,216495,False,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55309,925679,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,True
28504,476507,False,False,False,False,True,False,False,False,False,True,False,False,False,True,True,False
29047,486585,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False


In [29]:
items_ohe_df

,item_id,content_type_film,content_type_series,release_year_1897.0,release_year_1916.0,release_year_1917.0,release_year_1918.0,release_year_1920.0,release_year_1921.0,release_year_1922.0,...,keywords_490,keywords_491,keywords_492,keywords_493,keywords_494,keywords_495,keywords_496,keywords_497,keywords_498,keywords_499
iid,,,,,,,,,,,,,,,,,,,,,
3138,9853,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4858,15076,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
882,2904,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
490,1622,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.056351,0.0,0.0,0.0,0.0
2115,6677,True,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,1325,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5022,15610,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2045,6443,False,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [30]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

## Генератор и семплирование

Сделаем простой генератор. Он будет брать рандромного юзера, и два разных айтема - хороший пример и плохой:
- хорошим примером будет тот айтем, который был взят из датасета взаимодействий в соответствии с распределением просмотренных айтемов для этого юзера;
- а плохим айтемом будет просто любой другой _случайный айтем_*


In [31]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0]-1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


In [32]:
# инициализируем генератор
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                users=users_ohe_df.drop(["user_id"], axis=1), 
                interactions=interactions_vec)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 5303)
вектор 'хорошего' айтема: (1024, 9813)
вектор 'плохого' айтема: (1024, 9813)

вектор фичей юзера: (1024, 16)
вектор взаимодействий юзера с айтемами: (1024, 5303)


In [33]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (9813,)
USER_META_MODEL_SHAPE: (16,)
USER_INTERACTION_MODEL_SHAPE: (5303,)


In [34]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [35]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [36]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

In [37]:
# модель айтема
item_model().summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 9813)]       0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 128)          1256064     ['input_8[0][0]']                
                                                                                                  
 dense_8 (Dense)                (None, 128)          16384       ['dense_7[0][0]']                
                                                                                                  
 add_2 (Add)                    (None, 128)          0           ['dense_7[0][0]',                
                                                                  'dense_8[0][0]']          

In [38]:
# модель юзера
user_model().summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 dense_10 (Dense)               (None, 128)          2048        ['input_9[0][0]']                
                                                                                                  
 dense_12 (Dense)               (None, 128)          16384       ['dense_10[0][0]']               
                                                                                                  
 input_10 (InputLayer)          [(None, 5303)]       0           []                               
                                                                                            

In [39]:
# общая модель
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 5303)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 9813)]       0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 9813)]       0           []                               
                                                                                            

In [40]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                    users=users_ohe_df.drop(["user_id"], axis=1), 
                    interactions=interactions_vec,
                    batch_size=64), 
          steps_per_epoch=100, 
          epochs=30, 
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 278s 3s/step - loss: 0.3130 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 294s 3s/step - loss: 0.2332 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 294s 3s/step - loss: 0.2080 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 352s 4s/step - loss: 0.1973 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 270s 3s/step - loss: 0.1789 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 227s 2s/step - loss: 0.1748 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 241s 2s/step - loss: 0.1730 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 236s 2s/step - loss: 0.1697 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 403s 4s/step - loss: 0.1627 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 299s 3s/step - loss: 0.1548 - lr: 0.0010
Epoch 11/30
100/100 [============================

In [41]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid]
user_interaction_vec = interactions_vec[rand_uid]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), 
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 56ms/step


array([[1.7397815]], dtype=float32)

In [42]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

166/166 [==============================] - 0s 2ms/step


In [43]:
items_vecs.shape

(5303, 128)

In [44]:
users_meta_feats = users_ohe_df.drop(["user_id"], axis=1)
users_interaction_vec = interactions_vec

In [45]:
users_meta_feats.shape

(65552, 16)

In [46]:
users_interaction_vec.shape

(65552, 5303)

In [47]:
np.array(users_meta_feats).shape

(65552, 16)

In [48]:
users_vec = u2v.predict([np.array(users_meta_feats), 
                        np.array(users_interaction_vec)])

2049/2049 [==============================] - 8s 4ms/step


In [49]:
users_vec.shape

(65552, 128)

In [50]:
items_vecs.shape

(5303, 128)

In [51]:
dists = ED(users_vec, items_vecs)

In [52]:
dists.shape

(65552, 5303)

In [53]:
top10_iids = np.argsort(dists, axis=1)[:,:10]

In [54]:
top10_iids.reshape(dists.shape[0], 10).shape

(65552, 10)

In [55]:
top10_iids

array([[2418, 1705, 4134, ..., 3324, 4153, 5195],
       [3098, 4422, 4923, ..., 3577,  554, 3322],
       [4422, 1164, 3178, ..., 3098, 3748, 1507],
       ...,
       [1795, 2418, 1705, ..., 3815, 5226, 4914],
       [4098, 4574, 3098, ..., 2663,  387, 5172],
       [4923, 1783, 2020, ..., 4640, 4642, 3178]], dtype=int64)

In [56]:
top10_iids.shape

(65552, 10)

In [57]:
top10_iids_item = [iid_to_item_id[iid] for iid in top10_iids.reshape(-1)]

In [58]:
top10_iids_item = np.array(top10_iids_item).reshape(top10_iids.shape)

In [59]:
top10_iids_item.shape

(65552, 10)

In [60]:
df_dssm = pd.DataFrame(columns = ['user_id', 'item_id'])

In [61]:
df_dssm.head()

,user_id,item_id


In [62]:
df_dssm = pd.DataFrame({'user_id': [uid_to_user_id[uid] for uid in np.arange(top10_iids_item.shape[0])]})

In [63]:
df_dssm['item_id'] = list(top10_iids_item)

In [64]:
df_dssm

,user_id,item_id
0,2,"[7582, 5411, 12965, 13915, 5894, 16270, 1290, ..."
1,21,"[9728, 13865, 15297, 3734, 3935, 7417, 14703, ..."
2,53,"[13865, 3734, 9996, 15297, 4151, 5658, 1844, 9..."
3,60,"[15297, 13865, 142, 1844, 849, 3734, 7417, 104..."
4,81,"[5287, 1844, 14901, 14703, 8486, 7417, 142, 37..."
...,...,...
65547,1097486,"[9728, 3734, 13865, 142, 10440, 11237, 7417, 6..."
65548,1097489,"[3834, 10444, 7582, 15074, 3541, 14215, 9164, ..."
65549,1097508,"[5681, 7582, 5411, 12965, 10761, 10214, 15074,..."
65550,1097513,"[12841, 14264, 9728, 6455, 4436, 8821, 12501, ..."


In [65]:
df_dssm = df_dssm.explode('item_id')
df_dssm['rank'] = df_dssm.groupby('user_id').cumcount() + 1
df_dssm = df_dssm.groupby('user_id').agg({'item_id': list}).reset_index()

In [66]:
df_dssm.head()

,user_id,item_id
0,2,"[7582, 5411, 12965, 13915, 5894, 16270, 1290, ..."
1,21,"[9728, 13865, 15297, 3734, 3935, 7417, 14703, ..."
2,53,"[13865, 3734, 9996, 15297, 4151, 5658, 1844, 9..."
3,60,"[15297, 13865, 142, 1844, 849, 3734, 7417, 104..."
4,81,"[5287, 1844, 14901, 14703, 8486, 7417, 142, 37..."


In [67]:
result_dict = df_dssm.set_index('user_id')['item_id'].to_dict()

In [68]:
result_dict
with open('offline_dssm.pkl', 'wb') as f
    pickle.dump(result_dict, f)